In [9]:
!pip install yfinance
!pip install openai
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5

In [10]:
import yfinance as yf
import pandas as pd
import requests
import os
from google.colab import userdata
import gradio as gr
import openai

In [3]:
def get_company_overview(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    overview = {
        "company_name": info.get("longName"),
        "ticker": info.get("symbol"),
        "exchange": info.get("exchange"),
        "industry": info.get("industry"),
        "ceo": info.get("ceo") or info.get("companyOfficers")[0]['name'] if info.get("companyOfficers") else "N/A",
        "year_founded": info.get("startDate"),
        "headquarters": f"{info.get('city')}, {info.get('state')}" if info.get("city") and info.get("state") else "N/A",
        "description": info.get("longBusinessSummary")
    }
    return overview

def get_financial_metrics(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    metrics = {
        "market_cap": info.get("marketCap"),
        "total_revenue": info.get("totalRevenue"),
        "gross_profit_margin": info.get("grossMargins"),
        "ebitda_margin": info.get("ebitdaMargins"),
        "operating_margin": info.get("operatingMargins"),
        "net_profit_margin": info.get("profitMargins"),
        "eps_diluted": info.get("trailingEps"),
        "pe_ratio": info.get("trailingPE"),
        "forward_pe_ratio": info.get("forwardPE")
    }
    return metrics

def get_recent_news(ticker):
    stock = yf.Ticker(ticker)
    news_items = stock.news[:5]  # Get the latest 5 news articles
    news_list = []
    for item in news_items:
        news_list.append({
            "title": item.get("title"),
            "publisher": item.get("publisher"),
            "link": item.get("link"),
            "published_time": item.get("providerPublishTime")
        })
    return news_list

def get_analyst_estimates(ticker):
    stock = yf.Ticker(ticker)
    analysis = stock.recommendations
    # Process the DataFrame to extract estimates
    # This might require additional data sources or APIs
    estimates = {
        "revenue_estimates": {},
        "eps_estimates": {}
    }
    # Add logic to populate estimates
    return estimates

def get_stock_performance(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="1y")
    current_price = history['Close'][-1]
    performance = {
        "current_price": current_price,
        "52_week_range": (info.get("fiftyTwoWeekLow"), info.get("fiftyTwoWeekHigh")),
        "ytd_return": None,  # Calculate based on history
        "1y_total_return": None,
        "5y_total_return_cagr": None,
        "10y_total_return_cagr": None
    }
    # Add logic to calculate returns
    return performance

def get_full_stock_report(ticker):
    return {
        "overview": get_company_overview(ticker),
        "financial_metrics": get_financial_metrics(ticker),
        "recent_news": get_recent_news(ticker),
        "analyst_estimates": get_analyst_estimates(ticker),
        "stock_performance": get_stock_performance(ticker)
    }


In [4]:
get_full_stock_report("amd")

<ipython-input-3-a6c8635cf2f8>:61: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = history['Close'][-1]


{'overview': {'company_name': 'Advanced Micro Devices, Inc.',
  'ticker': 'AMD',
  'exchange': 'NMS',
  'industry': 'Semiconductors',
  'ceo': 'Dr. Lisa T. Su Ph.D.',
  'year_founded': None,
  'headquarters': 'Santa Clara, CA',
  'description': 'Advanced Micro Devices, Inc. operates as a semiconductor company worldwide. It operates through Data Center, Client, Gaming, and Embedded segments. The company offers x86 microprocessors and graphics processing units (GPUs) as an accelerated processing unit, chipsets, data center, and professional GPUs; and embedded processors, and semi-custom system-on-chip (SoC) products, microprocessor and SoC development services and technology, data processing unites, field programmable gate arrays (FPGA), and adaptive SoC products. It provides processors under the AMD Ryzen, AMD Ryzen PRO, Ryzen Threadripper, Ryzen Threadripper PRO, AMD Athlon, AMD Athlon PRO, and AMD PRO A-Series brand names; graphics under the AMD Radeon graphics and AMD Embedded Radeon

In [12]:
functions = [
    {
        "name": "get_full_stock_report",
        "description": "Get a detailed stock report.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "Stock ticker symbol."
                }
            },
            "required": ["ticker"]
        }
    },
    # Add individual functions if needed
]


In [13]:
openai.api_key = userdata.get('apopen')

In [16]:
def stock_chat(user_message):
    messages = [
        {"role": "system", "content": "You are a stock information bot. Provide detailed stock reports in the specified format."},
        {"role": "user", "content": user_message}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",  # Use the latest model that supports function calling
        messages=messages,
        functions=functions,
        function_call="auto"
    )

    response_message = response["choices"][0]["message"]

    if response_message.get("function_call"):
        function_name = response_message["function_call"]["name"]
        arguments = json.loads(response_message["function_call"]["arguments"])
        ticker = arguments.get("ticker")

        function_mapping = {
            "get_full_stock_report": get_full_stock_report,
            # Include other functions if necessary
        }

        function_to_call = function_mapping.get(function_name)
        if function_to_call:
            function_response = function_to_call(ticker)

            # Add the assistant's message and function response to the messages
            messages.append(response_message)
            messages.append({
                "role": "function",
                "name": function_name,
                "content": json.dumps(function_response)
            })

            # Get the final response from the assistant
            second_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-0613",
                messages=messages
            )

            return second_response['choices'][0]['message']['content']
        else:
            return "Sorry, I can't handle that request at the moment."
    else:
        return response_message.get('content', "Sorry, I didn't understand your request.")


In [17]:
messages = [
    {
        "role": "system",
        "content": """You are a stock information bot. Provide detailed stock reports in the following format:

[Company Name] ([Ticker]) Overview
Company Name: ...
Ticker: ...
Exchange: ...
Industry: ...
CEO: ...
Year Founded: ...
Headquarters: ...

Description: ...

Financial Metrics & Fundamentals
Market Cap: ...
Total Revenues: ...
Gross Profit Margin: ...
EBITDA Margin: ...
Operating Margin: ...
Net Profit Margin: ...
EPS Diluted: ...
P/E Ratio: ...
Forward P/E Ratio: ...

Recent News
- News Item 1
- News Item 2
- ...

Analyst Estimates
Revenue Estimates:
- 2024: ...
- 2025: ...
- 2026: ...

EPS Estimates:
- 2024: ...
- 2025: ...
- 2026: ...

Stock Performance
Current Price: ...
52-Week Range: ...
YTD Total Return: ...
1Y Total Return: ...
5Y Total Return CAGR: ...
10Y Total Return CAGR: ...

Summary
[Provide a brief summary of the company's performance and outlook.]
"""
    },
    {"role": "user", "content": user_message}
]

NameError: name 'user_message' is not defined

In [19]:
import pandas as pd
import openai
import gradio as gr
import requests
import yfinance as yf
import json
import os

# Set your OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

# Helper function to safely get data from dictionaries
def safe_get(data, key, default="N/A"):
    return data.get(key) if data.get(key) is not None else default

# Function to get company overview
def get_company_overview(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    overview = {
        "company_name": safe_get(info, "longName"),
        "ticker": safe_get(info, "symbol"),
        "exchange": safe_get(info, "exchange"),
        "industry": safe_get(info, "industry"),
        "ceo": safe_get(info, "ceo") or (info.get("companyOfficers")[0]['name'] if info.get("companyOfficers") else "N/A"),
        "year_founded": safe_get(info, "startDate"),
        "headquarters": f"{safe_get(info, 'city')}, {safe_get(info, 'state')}" if info.get("city") and info.get("state") else "N/A",
        "description": safe_get(info, "longBusinessSummary")
    }
    return overview

# Function to get financial metrics
def get_financial_metrics(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    metrics = {
        "market_cap": safe_get(info, "marketCap"),
        "total_revenue": safe_get(info, "totalRevenue"),
        "gross_profit_margin": safe_get(info, "grossMargins"),
        "ebitda_margin": safe_get(info, "ebitdaMargins"),
        "operating_margin": safe_get(info, "operatingMargins"),
        "net_profit_margin": safe_get(info, "profitMargins"),
        "eps_diluted": safe_get(info, "trailingEps"),
        "pe_ratio": safe_get(info, "trailingPE"),
        "forward_pe_ratio": safe_get(info, "forwardPE")
    }
    return metrics

# Function to get recent news
def get_recent_news(ticker):
    stock = yf.Ticker(ticker)
    news_items = stock.news[:5]  # Get the latest 5 news articles
    news_list = []
    for item in news_items:
        news_list.append({
            "title": item.get("title"),
            "publisher": item.get("publisher"),
            "link": item.get("link"),
            "published_time": item.get("providerPublishTime")
        })
    return news_list

# Function to get stock performance
def get_stock_performance(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="10y")
    current_price = history['Close'][-1]
    fifty_two_week_low = safe_get(info, "fiftyTwoWeekLow")
    fifty_two_week_high = safe_get(info, "fiftyTwoWeekHigh")

    # Calculate returns
    ytd_return = ((current_price - history['Close'][history.index >= f"{pd.Timestamp.now().year}-01-01"][0]) / history['Close'][history.index >= f"{pd.Timestamp.now().year}-01-01"][0]) * 100
    one_year_return = ((current_price - history['Close'][history.index == history.index[-252]][0]) / history['Close'][history.index == history.index[-252]][0]) * 100
    five_year_return = ((current_price - history['Close'][history.index == history.index[-1260]][0]) / history['Close'][history.index == history.index[-1260]][0]) * 100
    ten_year_return = ((current_price - history['Close'][0]) / history['Close'][0]) * 100

    performance = {
        "current_price": current_price,
        "52_week_range": f"{fifty_two_week_low} - {fifty_two_week_high}",
        "ytd_return": f"{ytd_return:.2f}%",
        "1y_total_return": f"{one_year_return:.2f}%",
        "5y_total_return_cagr": f"{(five_year_return/5):.2f}%",
        "10y_total_return_cagr": f"{(ten_year_return/10):.2f}%"
    }
    return performance

# Function to get full stock report
def get_full_stock_report(ticker):
    report = {
        "overview": get_company_overview(ticker),
        "financial_metrics": get_financial_metrics(ticker),
        "recent_news": get_recent_news(ticker),
        "stock_performance": get_stock_performance(ticker)
    }
    return report

# Function to format the stock report
def format_stock_report(report):
    overview = report.get('overview', {})
    financials = report.get('financial_metrics', {})
    news = report.get('recent_news', [])
    performance = report.get('stock_performance', {})

    formatted_report = f"""
**{overview.get('company_name')} ({overview.get('ticker')}) Overview**
- **Company Name:** {overview.get('company_name')}
- **Ticker:** {overview.get('ticker')}
- **Exchange:** {overview.get('exchange')}
- **Industry:** {overview.get('industry')}
- **CEO:** {overview.get('ceo')}
- **Year Founded:** {overview.get('year_founded')}
- **Headquarters:** {overview.get('headquarters')}

**Description:** {overview.get('description')}

**Financial Metrics & Fundamentals**
- **Market Cap:** {financials.get('market_cap')}
- **Total Revenues:** {financials.get('total_revenue')}
- **Gross Profit Margin:** {financials.get('gross_profit_margin')}
- **EBITDA Margin:** {financials.get('ebitda_margin')}
- **Operating Margin:** {financials.get('operating_margin')}
- **Net Profit Margin:** {financials.get('net_profit_margin')}
- **EPS Diluted:** {financials.get('eps_diluted')}
- **P/E Ratio:** {financials.get('pe_ratio')}
- **Forward P/E Ratio:** {financials.get('forward_pe_ratio')}

**Recent News**
"""
    for item in news:
        formatted_report += f"- [{item.get('title')}]({item.get('link')}) ({item.get('publisher')})\n"

    formatted_report += f"""
**Stock Performance**
- **Current Price:** {performance.get('current_price')}
- **52-Week Range:** {performance.get('52_week_range')}
- **YTD Total Return:** {performance.get('ytd_return')}
- **1Y Total Return:** {performance.get('1y_total_return')}
- **5Y Total Return CAGR:** {performance.get('5y_total_return_cagr')}
- **10Y Total Return CAGR:** {performance.get('10y_total_return_cagr')}

**Summary**
*Provide a brief summary of the company's performance and outlook.*
"""
    return formatted_report

# Define the function schema for OpenAI's function calling
functions = [
    {
        "name": "get_full_stock_report",
        "description": "Get a detailed stock report.",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker": {
                    "type": "string",
                    "description": "Stock ticker symbol."
                }
            },
            "required": ["ticker"]
        }
    }
]

# Function to handle the chat and function calling
def stock_chat(user_message):
    messages = [
        {
            "role": "system",
            "content": """You are a stock information bot. Provide detailed stock reports in the following format:

[Company Name] ([Ticker]) Overview
Company Name: ...
Ticker: ...
Exchange: ...
Industry: ...
CEO: ...
Year Founded: ...
Headquarters: ...

Description: ...

Financial Metrics & Fundamentals
Market Cap: ...
Total Revenues: ...
Gross Profit Margin: ...
EBITDA Margin: ...
Operating Margin: ...
Net Profit Margin: ...
EPS Diluted: ...
P/E Ratio: ...
Forward P/E Ratio: ...

Recent News
- News Item 1
- News Item 2
- ...

Stock Performance
Current Price: ...
52-Week Range: ...
YTD Total Return: ...
1Y Total Return: ...
5Y Total Return CAGR: ...
10Y Total Return CAGR: ...

Summary
[Provide a brief summary of the company's performance and outlook.]
"""
        },
        {"role": "user", "content": user_message}
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",  # Use the latest model that supports function calling
        messages=messages,
        functions=functions,
        function_call="auto",
        temperature=0
    )

    response_message = response["choices"][0]["message"]

    if response_message.get("function_call"):
        function_name = response_message["function_call"]["name"]
        arguments = json.loads(response_message["function_call"]["arguments"])
        ticker = arguments.get("ticker")

        function_mapping = {
            "get_full_stock_report": get_full_stock_report
        }

        function_to_call = function_mapping.get(function_name)
        if function_to_call:
            function_response = function_to_call(ticker)

            # Format the report
            formatted_response = format_stock_report(function_response)

            # Add the assistant's message and function response to the messages
            messages.append(response_message)
            messages.append({
                "role": "function",
                "name": function_name,
                "content": formatted_response
            })

            # Get the assistant's final response
            final_response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-0613",
                messages=messages,
                temperature=0
            )

            return final_response['choices'][0]['message']['content']
        else:
            return "Sorry, I can't handle that request at the moment."
    else:
        return response_message.get('content', "Sorry, I didn't understand your request.")

# Gradio Interface
iface = gr.Interface(
    fn=stock_chat,
    inputs=gr.Textbox(lines=2, placeholder="Ask me about stock info like - 'Provide a detailed report on AMZN'", label="Stock Queries"),
    outputs=gr.Markdown(label="Stock Report"),
    title="Stock Information Assistant By Parvez Alam",
    description="Ask me about stock prices and financial data.",
    theme="default"
)

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8233c461332592b15d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
